# Guided Exercise: Fairness

### Goals 🎯
In this tutorial, you will view the fairness test results from part 1, identify the root cause and mitigate the fairness issues in the model.

The data used is ACS Employment data made available through the [*folktables* repository](https://github.com/zykls/folktables)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/truera-examples/blob/release/prod/starter-examples/starter-fairness-part-2.ipynb)

### First, set the credentials for your TruEra deployment.
If you don't have credentials yet, get them instantly by signing up for free at: https://www.truera.com

In [ ]:
# connection details
TRUERA_URL = "https://app.truera.net/"
AUTH_TOKEN = "..."

### Install required packages for running in colab

In [ ]:
! pip install truera==12.1.0 pandas==1.5.2 sklearn==1.3.0 xgboost==1.6.2 smart-open==6.3.0

### From here, you can run the rest of the notebook to follow the analysis.

In [ ]:
import logging

import pandas as pd
import sklearn.metrics
import xgboost as xgb
from sklearn import preprocessing
from sklearn.utils import resample
from truera.client.ingestion import ColumnSpec, ModelOutputContext
from truera.client.truera_authentication import TokenAuthentication
from truera.client.truera_workspace import TrueraWorkspace

auth = TokenAuthentication(AUTH_TOKEN)
tru = TrueraWorkspace(TRUERA_URL, auth)

In [ ]:
# create the first project and data collection
project_name = "Starter Example Companion - Fairness"
tru.set_project(project_name)
tru.set_data_collection("Data Collection v1")
tru.get_models()

In [ ]:
tru.set_model(tru.get_models()[0])
tru.tester.get_model_test_results(test_types=["fairness"])

* What? Shown in the model test results, the first version of the test fails the Impact Ratio Test.

* Why? After exploring results, CA has a better impact ratio than in NY, leading us to a hypothesis that CA might have a lower difference in ground truth outcomes between men and women. This phenomenon can also be referred to as *dataset disparity*.

* We can examine the dataset disparity either in the web app or in the SDK (shown below).

In [ ]:
# dataset disparity for 2014-NY
explainer = tru.get_explainer("2014-NY")
explainer.set_segment(segment_group_name="Sex", segment_name="Female")
mean_outcome_females_NY_2014 = explainer.get_ys().mean()
explainer.set_segment(segment_group_name="Sex", segment_name="Male")
mean_outcome_males_NY_2014 = explainer.get_ys().mean()
print("2014-NY dataset disparity: " + str(mean_outcome_females_NY_2014 - mean_outcome_males_NY_2014))

# dataset disparity for 2014-CA
explainer = tru.get_explainer("2014-CA")
explainer.set_segment(segment_group_name="Sex", segment_name="Female")
mean_outcome_females_CA_2014 = explainer.get_ys().mean()
explainer.set_segment(segment_group_name="Sex", segment_name="Male")
mean_outcome_males_CA_2014 = explainer.get_ys().mean()
print("2014-CA dataset disparity: " + str(mean_outcome_females_CA_2014 - mean_outcome_males_CA_2014))

This calculation of dataset disparity for NY and CA confirm our hypothesis.

Going forward, we should retrain our model on CA with the belief that more fair training data will lead to a more fair model. But let's not stop there.

* We notice that sex is included in the training data, and the second leading contributor to disparity. We should remove it so it's not learned by the model.

* What next? Let's see the fairness outcome after removing sex from the training data and using CA as our training set.

In [ ]:
# get data and feature map

from smart_open import open

data_s3_file_name = "https://truera-examples.s3.us-west-2.amazonaws.com/data/starter-fairness/starter-fairness-data-compressed.pickle"
with open(data_s3_file_name, "rb") as f:
    data = pd.read_pickle(f)

feature_map_s3_file_name = "https://truera-examples.s3.us-west-2.amazonaws.com/data/starter-fairness/starter-fairness-feature-map.pickle"
with open(feature_map_s3_file_name, "rb") as f:
    feature_map = pd.read_pickle(f)
feature_map.pop("Sex")

tru.add_data_collection("Data Collection v2", pre_to_post_feature_map=feature_map, provide_transform_with_model=False)

# add data splits to the collection we just created
year_begin = 2014
year_end = 2016  # exclusive
states = ["CA", "NY"]

for year in range(year_begin, year_end):
    for state in states:
        data_split_name = f"{year}-{state}"
        print(f"Ingesting data-split: {data_split_name}...")
        ids = [f"{i}" for i in range(len(data[year][state]["data_preprocessed"]))]
        data[year][state]["data_preprocessed"]["id"] = ids
        data[year][state]["data_postprocessed"]["id"] = ids
        data[year][state]["label"] = pd.DataFrame(data[year][state]["label"])
        data[year][state]["label"]["id"] = ids
        data[year][state]["extra_data"] = pd.DataFrame(data[year][state]["extra_data"][["LANX"]])
        data[year][state]["extra_data"]["id"] = ids
        # data
        merged_data = data[year][state]["data_preprocessed"].\
            merge(data[year][state]["data_postprocessed"]).\
            merge(data[year][state]["label"]).\
            merge(data[year][state]["extra_data"])
        tru.add_data(
            data=merged_data,
            data_split_name=data_split_name,
            column_spec=ColumnSpec(
                id_col_name="id",
                pre_data_col_names=list(data[year][state]["data_preprocessed"].columns.drop(
                    ["id", "Sex"])),  # drop sex from pre-data
                post_data_col_names=list(data[year][state]["data_postprocessed"].columns.drop(
                    ["id", "Sex_Male", "Sex_Female"])),
                label_col_names=list(data[year][state]["label"].columns.drop("id")),
                extra_data_col_names=list(data[year][state]["extra_data"].columns.drop("id")) +
                ["Sex"]  # add sex to extra-data
            ))
tru.set_influences_background_data_split(f"{year_begin}-{states[0]}")

In [ ]:
# train xgboost
models = {}
model_name_v2 = "model_2"

models[model_name_v2] = xgb.XGBClassifier(eta=0.2, max_depth=4)

models[model_name_v2].fit(data[2014]["CA"]["data_postprocessed"].drop(["Sex_Female", "Sex_Male", "id"], axis=1),
                          data[2014]["CA"]["label"].drop("id", axis=1))

train_params = {"model_type": "xgb.XGBClassifier", "eta": 0.2, "max_depth": 4}

# ingest the model
tru.set_project(project_name)
tru.set_data_collection("Data Collection v2")
tru.add_python_model(model_name_v2, models[model_name_v2], train_split_name="2014-CA", train_parameters=train_params)

In [ ]:
tru.set_model("model_2")
tru.set_influences_background_data_split("2014-NY")

# set model output context
model_output_context = ModelOutputContext(model_name=model_name_v2,
                                          score_type="probits",
                                          background_split_name=f"2014-NY",
                                          influence_type='tree-shap-interventional')

for year in range(year_begin, year_end):
    for state in states:
        data_split_name = f"{year}-{state}"
        tru.set_data_split(data_split_name)
        print(f"Ingesting data-split: {data_split_name}...")
        preds = tru.get_ys_pred().reset_index(names="id")
        # predictions
        tru.add_data(data=preds,
                     data_split_name=data_split_name,
                     column_spec=ColumnSpec(id_col_name="id", prediction_col_names="__truera_prediction__"))
        # influences
        tru.compute_feature_influences()

In [ ]:
# view model test results
tru.set_model(model_name_v2)
tru.tester.get_model_test_results(test_types=["fairness"])

* What? Our newest model now passes 2/4 impact ratio tests.
* Shown on the Dataset Disparity section of the Fairness page in the web app, we notice that there is a lower positive outcome rate for females in the data. Let's check the performance of females with a positive label.

In [ ]:
tru.set_model(model_name_v2)
tru.set_data_split("2014-CA")
tru.add_segment_group(
    "Sex + Label", {
        "Other": "(Sex == 'Male') OR (_DATA_GROUND_TRUTH == 0)",
        "Female + Label 1": "(Sex == 'Female') AND (_DATA_GROUND_TRUTH == 1)"
    })
explainer = tru.get_explainer(base_data_split="2014-CA")

In [ ]:
explainer.set_segment(segment_group_name="Sex + Label", segment_name="Female + Label 1")
print("Female + label 1 performance: " + str(explainer.compute_performance(metric_type="CLASSIFICATION_ACCURACY")))
explainer.set_segment(segment_group_name="Sex + Label", segment_name="Other")
print("Other performance: " + str(explainer.compute_performance(metric_type="CLASSIFICATION_ACCURACY")))

Our hunch was correct. To correct for this, rebalance the training set by overampling female with label==1 in train set because female with label==1 has more error than rest_of_pop with label==1

In [ ]:
def rebalance_gender(df, data_type):
    if data_type == 0:
        df_female_true = df[(df["Sex"] == "Female") & (df["PINCP"] == True)]
        df_else = df[~((df["Sex"] == "Female") & (df["PINCP"] == True))]
    else:
        df_female_true = df[(df["Sex_Female"] == 1) & (df["PINCP"] == True)]
        df_else = df[~((df["Sex_Female"] == 1) & (df["PINCP"] == True))]

    if data_type == 0:
        num_samples = len(df[(df["Sex"] == "Male") & (df["PINCP"] == True)])
    else:
        num_samples = len(df[(df["Sex_Male"] == 1) & (df["PINCP"] == True)])
    # Resample female target segment so that they are the same size as male
    df_female_true_resampled = resample(
        df_female_true,
        replace=True,
        n_samples=num_samples,
        random_state=1  # include random seed so we can perform same sampling on each data set
    )

    return pd.concat([df_female_true_resampled, df_else])

In [ ]:
data[2014]["CA"]["data_preprocessed_resampled"] = rebalance_gender(data[2014]["CA"]["data_preprocessed"].reset_index(drop=True).\
                                        merge(data[2014]["CA"]["label"].reset_index(drop=True), on="id"), 0).drop(["Sex","PINCP"], axis=1)
data[2014]["CA"]["data_postprocessed_resampled"] = rebalance_gender(data[2014]["CA"]["data_postprocessed"].reset_index(drop=True).\
                                        merge(data[2014]["CA"]["label"].reset_index(drop=True), on="id"), 1).drop(["Sex_Male","Sex_Female", "PINCP"], axis=1)
data[2014]["CA"]["label_resampled"] = rebalance_gender(pd.DataFrame(data[2014]["CA"]["label"].reset_index(drop=True)).\
                                        merge(data[2014]["CA"]["data_preprocessed"].reset_index(drop=True), on="id"), 0).drop(["Sex"], axis=1)[["PINCP","id"]]
data[2014]["CA"]["extra_data_resampled"] = rebalance_gender(pd.DataFrame(data[2014]["CA"]["extra_data"].reset_index(drop=True)).\
                                        merge(data[2014]["CA"]["data_preprocessed"].reset_index(drop=True), on="id").\
                                        merge(data[2014]["CA"]["label"].reset_index(drop=True), on="id"), 0)[["LANX","id","Sex"]]

ids = [f"{i}" for i in range(len(data[2014]["CA"]["data_preprocessed_resampled"]))]
data[2014]["CA"]["data_preprocessed_resampled"]["id"] = ids
data[2014]["CA"]["data_postprocessed_resampled"]["id"] = ids
data[2014]["CA"]["label_resampled"]["id"] = ids
data[2014]["CA"]["extra_data_resampled"]["id"] = ids

In [ ]:
merged_data = data[2014]["CA"]["data_preprocessed_resampled"].\
            merge(data[2014]["CA"]["data_postprocessed_resampled"]).\
            merge(data[2014]["CA"]["label_resampled"]).\
            merge(data[2014]["CA"]["extra_data_resampled"])
tru.add_data(data=merged_data,
             data_split_name="2014-CA-resampled",
             column_spec=ColumnSpec(
                 id_col_name="id",
                 pre_data_col_names=list(data[2014]["CA"]["data_preprocessed_resampled"].columns.drop(["id"])),
                 post_data_col_names=list(data[2014]["CA"]["data_postprocessed_resampled"].columns.drop(["id"])),
                 label_col_names=list(data[2014]["CA"]["label_resampled"].columns.drop("id")),
                 extra_data_col_names=list(data[2014]["CA"]["extra_data_resampled"].columns.drop("id"))))

In [ ]:
# train xgboost
model_name_v3 = "model_3"

models[model_name_v3] = xgb.XGBClassifier(eta=0.2, max_depth=4)
models[model_name_v3].fit(data[2014]["CA"]["data_postprocessed_resampled"].drop("id", axis=1),
                          data[2014]["CA"]["label_resampled"].drop("id", axis=1))

# ingest the model
tru.add_python_model(model_name_v3,
                     models[model_name_v3],
                     train_split_name="2014-CA-resampled",
                     train_parameters=train_params)

In [ ]:
tru.set_model(model_name_v3)
tru.set_influences_background_data_split("2014-CA-resampled")

# set model output context
model_output_context = ModelOutputContext(model_name=model_name_v3,
                                          score_type="probits",
                                          background_split_name=f"2014-CA-resampled",
                                          influence_type='tree-shap-interventional')

data_split_name = "2014-CA-resampled"
tru.set_data_split(data_split_name)
print(f"Ingesting data-split: 2014-CA-resampled...")

# predictions
preds = tru.get_ys_pred().reset_index(names="id")
tru.add_data(data=preds,
             data_split_name=data_split_name,
             column_spec=ColumnSpec(id_col_name="id", prediction_col_names="__truera_prediction__"))

# influences
tru.compute_feature_influences()

In [ ]:
for year in range(year_begin, year_end):
    for state in states:
        data_split_name = f"{year}-{state}"
        tru.set_data_split(data_split_name)
        print(f"Ingesting data-split: {data_split_name}...")
        preds = tru.get_ys_pred().reset_index(names="id")
        # predictions
        tru.add_data(data=preds,
                     data_split_name=data_split_name,
                     column_spec=ColumnSpec(id_col_name="id", prediction_col_names="__truera_prediction__"))
        # influences
        tru.compute_feature_influences()

In [ ]:
# view model test results
tru.set_model(model_name_v3)
tru.tester.get_model_test_results(test_types=["fairness"])

### After rebalancing, the model passes 4/4 impact ratio tests, giving us confidence in the fairness of the model.